# Data Exploratory Analysis
This notebook contains my initial exploratory analysis of the presented data, alongside my thought process for each block of code.


### Initial Data Exploration 
I went through the data and inspected it from a few different angles to get a better sense of what I was working with, and what possible patterns could be used for analysis.

In [0]:
# Ingestion of Data into Accounts and Commitments Dataframes
accounts_df = spark.table("workspace.default.accounts")
commitments_df = spark.table("workspace.default.commitments")

In [0]:
# Show schema to understand data types
print("Accounts schema:")
accounts_df.printSchema()

print("Commitments schema:")
commitments_df.printSchema()


Accounts schema:
root
 |-- id: string (nullable = true)
 |-- Organization_name_hashed: string (nullable = true)
 |-- RelationshipType: string (nullable = true)
 |-- AccountType: string (nullable = true)
 |-- BiteLow: double (nullable = true)
 |-- BiteHigh: double (nullable = true)
 |-- AccountLead: string (nullable = true)
 |-- HQCityStateCountry: string (nullable = true)

Commitments schema:
root
 |-- id: string (nullable = true)
 |-- Commitment: double (nullable = true)
 |-- VehicleType: string (nullable = true)
 |-- LimitedPartner: string (nullable = true)
 |-- FundName: string (nullable = true)



Checking Null and NaN counts in accounts and committments. It's important to understand the gaps in our data, especially when creating ML models and drawing conclusions from them.

In [0]:
from pyspark.sql.functions import col, count, when, isnan
from pyspark.sql.types import StringType, DoubleType, IntegerType, FloatType, LongType, DecimalType

def count_nulls(df):
    exprs = []
    for c, t in df.dtypes:
        if t in ('double', 'float', 'int', 'bigint', 'long', 'decimal'):
            # For numeric types, check both null and NaN
            exprs.append(count(when(col(c).isNull() | isnan(col(c)), c)).alias(c))
        else:
            # For non-numeric, just check nulls
            exprs.append(count(when(col(c).isNull(), c)).alias(c))
    return df.select(exprs)

print("Null/NaN counts in accounts:")
count_nulls(accounts_df).show()

print("Null/NaN counts in commitments:")
count_nulls(commitments_df).show()


Null/NaN counts in accounts:
+---+------------------------+----------------+-----------+-------+--------+-----------+------------------+
| id|Organization_name_hashed|RelationshipType|AccountType|BiteLow|BiteHigh|AccountLead|HQCityStateCountry|
+---+------------------------+----------------+-----------+-------+--------+-----------+------------------+
|  0|                       0|               0|        187|   2567|    2568|       1485|              1256|
+---+------------------------+----------------+-----------+-------+--------+-----------+------------------+

Null/NaN counts in commitments:
+---+----------+-----------+--------------+--------+
| id|Commitment|VehicleType|LimitedPartner|FundName|
+---+----------+-----------+--------------+--------+
|  0|        15|         32|          1004|      24|
+---+----------+-----------+--------------+--------+



Notably, there is a quite large number of NaN values in BiteLow, BiteHigh, AccountLead, Location in Accounts as well as the LimitedPartner Assignment in Commitments.

We can also analyze the 'orphaned' LPs, who have Commitments but aren't present in Accounts.

In [0]:
from pyspark.sql import functions as F

# Orphaned LPs = commitments with NO matching accounts.id
orphaned_commitments_df = (
    commitments_df.alias("c")
    .join(
        accounts_df.select("id").alias("a"),
        F.col("c.LimitedPartner") == F.col("a.id"),
        "left_anti"
    )
    .filter(F.col("c.LimitedPartner").isNotNull())
)

orphan_count = orphaned_commitments_df.count()
print(f"Orphaned LP commitments (excluding NULL LP): {orphan_count}")


Orphaned LP commitments (excluding NULL LP): 679


There are over 600 'orphaned' LPs in the dataset. This is pretty significant, and suggests there are some major gaps in the accounts data. In a production environment we would need to re-evaluate our data but since we are using sample data, we'll have to work with it.

In [0]:
# Display first 5 non-null BiteLow and BiteHigh from accounts_df
display(
    accounts_df.filter(
        (accounts_df.BiteLow.isNotNull()) &
        (accounts_df.BiteHigh.isNotNull())
    )
    .select("BiteLow", "BiteHigh")
    .limit(5)
)

# Display first 5 non-null Commitment from commitments_df
display(
    commitments_df.filter(
        commitments_df.Commitment.isNotNull()
    )
    .select("Commitment")
    .limit(5)
)

BiteLow,BiteHigh
100.0,200.0
10.0,30.0
5.0,20.0
5.0,20.0
15.0,45.0


Commitment
120000.0
5.0E7
1750000.0
3.24926E7
975000.0


One thing to note is that BiteHigh/Low are all represented in 'millions' whereas Commitments are in raw data, so we will have to scale BiteHigh/Low for better reporting.

Next, I want to analyze the unique value counts for the RelationshipType, AccountType, and VehicleType Columns, as that will allow us to understand where these prospective investers are coming from. We can also visualize through a pie chart.

In [0]:
# Understand Value Counts
accounts_df.groupBy("RelationshipType") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(150, truncate=False)

accounts_df.groupBy("AccountType") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(50,truncate=False)

commitments_df.groupBy("VehicleType") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(50,truncate=False)



+------------------------------------------------------------------------------------------------------------+-----+
|RelationshipType                                                                                            |count|
+------------------------------------------------------------------------------------------------------------+-----+
|Other Partner / Enabler                                                                                     |562  |
|Need Introduction                                                                                           |528  |
|Friends & Family, Limited Partner                                                                           |509  |
|Friends & Family                                                                                            |401  |
|Low Prospect                                                                                                |342  |
|High Prospect                                                  

In [0]:
# Plot Pie Charts

import plotly.express as px
import pandas as pd

def plot_pie_plotly_with_other(df, category_col, count_col='count', top_n=10, title=""):
    # Sort and create 'Other' category
    df_sorted = df.sort_values(by=count_col, ascending=False).reset_index(drop=True)
    top_categories = df_sorted.iloc[:top_n]
    other_categories = df_sorted.iloc[top_n:]
    other_sum = other_categories[count_col].sum()
    
    # Append 'Other' row if there is any data in other categories
    if other_sum > 0:
        df_final = pd.concat([top_categories, pd.DataFrame({category_col: ['Other'], count_col: [other_sum]})], ignore_index=True)
    else:
        df_final = top_categories

    fig = px.pie(df_final, values=count_col, names=category_col, title=title, hole=0.3)  # hole=0.3 for donut chart
    
    # Add border to slices for better visuals
    fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
    
    fig.show()

# Example usage for RelationshipType
rel_counts = accounts_df.groupBy("RelationshipType").count().toPandas()
plot_pie_plotly_with_other(rel_counts, 'RelationshipType', 'count', top_n=10, title='RelationshipType Distribution')

# AccountType
acct_counts = accounts_df.groupBy("AccountType").count().toPandas()
plot_pie_plotly_with_other(acct_counts, 'AccountType', 'count', top_n=10, title='AccountType Distribution')

# VehicleType
veh_counts = commitments_df.groupBy("VehicleType").count().toPandas()
plot_pie_plotly_with_other(veh_counts, 'VehicleType', 'count', top_n=10, title='VehicleType Distribution')


We can already indentify some trends in the data. The Main Fund is overwhelmingly the fund with the largest # of commitments, "High Net Worth Individuals" and "Other" categories tend to make up the largest Account Types. The RelationshipType distribution is fairly evenly distributed.

Next, I want to analyze the makeup of the BiteLow and BiteHigh columns to get a better understanding of the average, mean, and standard deviation.

In [0]:
accounts_df.describe("BiteLow", "BiteHigh").show()
commitments_df.describe("Commitment").show()


+-------+------------------+------------------+
|summary|           BiteLow|          BiteHigh|
+-------+------------------+------------------+
|  count|              1420|              1419|
|   mean|14.587922535211266|40.882840028188866|
| stddev| 26.66185688703746|  79.3028969222704|
|    min|               0.0|               0.0|
|    max|             500.0|            1000.0|
+-------+------------------+------------------+

+-------+--------------------+
|summary|          Commitment|
+-------+--------------------+
|  count|                8067|
|   mean|   5772520.951178878|
| stddev|1.4192247973811848E7|
|    min|                 0.0|
|    max|               3.3E8|
+-------+--------------------+



In [0]:
from pyspark.sql.functions import col

joined_df=spark.table("default.prod_accounts_commitments")
filtered_df = joined_df.filter(
    (col("BiteLow").isNotNull()) & (col("BiteLow") > 0) &
    (col("BiteHigh").isNotNull()) & (col("BiteHigh") > 0) &
    (col("Commitment").isNotNull()) & (col("Commitment") > 0)
)
pdf = filtered_df.select("BiteLow", "BiteHigh", "Commitment").toPandas()


To get a deeper understanding of the VehicleType field, we can get an understanding of Average, Total, and Count of commitments for each fund.

In [0]:
from pyspark.sql.functions import mean, sum

commitments_df.groupBy("VehicleType").agg(
    count("Commitment").alias("count_commitments"),
    mean("Commitment").alias("avg_commitment"),
    sum("Commitment").alias("total_commitment")
).orderBy("total_commitment", ascending=False).show()


+--------------------+-----------------+--------------------+--------------------+
|         VehicleType|count_commitments|      avg_commitment|    total_commitment|
+--------------------+-----------------+--------------------+--------------------+
|           Main Fund|             4558|   8860678.666377798|4.038697336135000...|
|              Q Fund|             1470|  1090797.3010748301|1.6034720325800002E9|
|         Annual Fund|              113| 1.284536358477876E7|     1.45152608508E9|
|                 SMA|               11| 8.606025909181818E7|      9.4666285001E8|
|              B Fund|              763|   945705.1651114024|      7.2157304098E8|
|Annual Fund - Cot...|              464|  1257877.1551724137|           5.83655E8|
|           Co-Invest|               93|  3625399.9411827954|      3.3716219453E8|
|                NULL|               23|1.2411224347826088E7|         2.8545816E8|
|Annual Fund - Kir...|               96|   981510.4166666666|            9.4225E7|
|   

We can see that different funds have different strategies. The Main Fund has a lower avg commitment but a higher count of commitments, while the SMA Fund has by far the highest avg commitment but only 11 commitments total.

In [0]:
joined_df = spark.table("default.prod_accounts_commitments")
from pyspark.sql.functions import count, avg, sum
joined_df.groupBy("RelationshipType").agg(
    count("Commitment").alias("num_commitments"),
    avg("Commitment").alias("avg_commitment"),
    sum("Commitment").alias("total_commitment")
).orderBy("total_commitment", ascending=False).show(50)


+--------------------+---------------+--------------------+--------------------+
|    RelationshipType|num_commitments|      avg_commitment|    total_commitment|
+--------------------+---------------+--------------------+--------------------+
|     Limited Partner|           4270|   8410634.491606556|3.591340927915999...|
|Limited Partner, ...|            420|   9380452.138952384|3.9397898983600016E9|
|             unknown|           1674|  1462546.6571146958|2.4483031040100007E9|
|a16z Vehicle, Lim...|             75|      2.1331303584E7|      1.5998477688E9|
|Friends & Family,...|           1027|   799228.5959006825|  8.20807767990001E8|
|Limited Partner, ...|             28|2.8826236931428574E7|      8.0713463408E8|
|    a16z Employee LP|            418|   1456072.120741627|      6.0863814647E8|
|           Exited LP|              8|       2.425189394E7|      1.9401515152E8|
|Limited Partner, ...|              1|               5.0E7|               5.0E7|
|        a16z Vehicle|      

Looking into the RelationshipType details, we can identify similar trends in the data. The general LP category is the largest with a lower avg commitment, while certain other Relationship Types have much higher average commitments. The "unknown" field is very large, which indicates a data quality issue that's worth prioritizing for more accurate analysis.

Let's analyze the last categorical category, AccountType.

In [0]:
from pyspark.sql.functions import count, avg, sum

# Aggregate by AccountType
joined_df.groupBy("AccountType").agg(
    count("Commitment").alias("num_commitments"),
    avg("Commitment").alias("avg_commitment"),
    sum("Commitment").alias("total_commitment")
).orderBy("total_commitment", ascending=False).show(50, truncate=False)


+-------------------------+---------------+--------------------+--------------------+
|AccountType              |num_commitments|avg_commitment      |total_commitment    |
+-------------------------+---------------+--------------------+--------------------+
|Fund of Funds            |1473           |5845379.906693823   |8.610244602560001E9 |
|Endowment                |683            |1.2520406265139092E7|8.55143747909E9     |
|Single Family Office     |806            |5951756.8479280425  |4.797116019430002E9 |
|Sovereign Wealth Fund    |95             |3.7316803588421054E7|3.5450963409E9      |
|Corporate Pension        |145            |2.1635717629448276E7|3.13717905627E9     |
|Foundation               |351            |8009589.4369800575  |2.81136589238E9     |
|Public Pension           |90             |3.0705005991111107E7|2.7634505392E9      |
|unknown                  |1674           |1462546.6571146958  |2.4483031040100007E9|
|a16z Vehicle             |496            |4539253.708

The Account Type with the most commitments is 'Unknown', which similarly poses a problem for very granular analysis. However, we can still inspect the other features in the data. While Fund of Funds has the largest total commitment, some small groups like Corporate Strategic and Healthcare Systems have a surprisingly high average commitment, across a quite small number of overall commitments. These may be under-explored segments.